<a href="https://colab.research.google.com/github/shalinisaini/CS60092_9A_18CS60R47/blob/master/DL_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
SEED =42
torch.manual_seed(SEED)

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 


In [111]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:

url = 'https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Test_5K.csv'
df1 = pd.read_csv(url, sep='\t')

In [0]:
url = 'https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Train_20K.csv'
df2 = pd.read_csv(url, sep='\t')

In [0]:
train_data = df2['text'].astype(str).tolist()
train_label = df2['label'].astype(int)
test_data = df1['text'].astype(str).tolist()
test_label = df1['label'].astype(int)

train_data = [x.lower() for x in train_data]
test_data = [x.lower() for x in test_data]


 


In [0]:
def generate_word_ids(dataset):
  word_to_ix = {}
  print(dataset)
  for sent in dataset:
      for word in sent.split():
          if word not in word_to_ix:
              word_to_ix[word] = 1
          else:
              word_to_ix[word] = word_to_ix[word]+1
  return word_to_ix

In [131]:
word_to_ix = generate_word_ids(train_data+test_data)

NUM_LABELS = 2

list=[]
for i in word_to_ix:
  if(word_to_ix[i]<10):
    list.append(i)
word_to_ix={}
for i in list:
  word_to_ix[i] = len(word_to_ix)
  
VOCAB_SIZE = len(word_to_ix)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
train=[]
test=[]
for i in test_data:
  test.append(i.split(" "))
for i in train_data:
  train.append(i.split(" "))
train_data = train
test_data = test

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 50)
    #self.lin2 = nn.Linear(50, 50)
    #self.lin3 = nn.Linear(50, 50)
    self.lin4 = nn.Linear(50, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin4(self.lin(x)))

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        if word not in word_to_ix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            c=1
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])


In [0]:
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE)


In [0]:
def accuracy(data,label):
  count=0
  for idx in range(len(data)):
      bow_vec = Variable(make_bow_vector(data[idx], word_to_ix))
      labels = Variable(torch.LongTensor(label[idx]))
      logprobs = bow(bow_vec)
      pred = np.argmax(logprobs.data.numpy())
      #print(pred,label[idx]+1)
      if(pred == label[idx]):
        count+=1
  print("accuracy",count/len(data))
        


In [0]:
# define a loss function and an optimizer
loss_function = nn.MSELoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [140]:
# the training loop
for epoch in range(5):
    print("epoch",epoch)
    for idx in range(len(train_data)):
        # get the training data
        bow.zero_grad()
        #print(train_data[idx])
        #print(train_label[idx])
        bow_vec = Variable(make_bow_vector(train_data[idx], word_to_ix))
        label  = Variable(torch.FloatTensor(1+train_label[idx]))
        #print(bow_vec,label,train_label[idx])
        probs = bow(bow_vec) # forward pass
        #print(probs)
        loss = loss_function(probs, label)
        loss.backward()
        opt.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    accuracy(train_data,train_label)


epoch 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


CURRENT LOSS: 0.5
accuracy 0.4996944274681927
epoch 1
CURRENT LOSS: 0.5
accuracy 0.4996944274681927
epoch 2
CURRENT LOSS: 0.5
accuracy 0.4996944274681927
epoch 3
CURRENT LOSS: 0.5
accuracy 0.4996944274681927
epoch 4
CURRENT LOSS: 0.5
accuracy 0.4996944274681927


In [0]:
accuracy(test_data,test_label)

In [0]:
torch.save(bow,'Task1A.mdl')
from google.colab import files
files.download("Task1A.mdl")